# 🚀 Vector-Powered Database Query Agent

## 📋 Overview
This notebook demonstrates building an intelligent database query agent that uses **semantic search** to understand natural language questions and find relevant database tables before executing SQL queries.



## 🎯 Key Components
1. **Chroma Vector Store**: Indexes database schemas for fast semantic search
2. **Embedding Model**: Converts table schemas to vector representations
3. **Custom Tools**: Semantic search tools for table discovery
4. **ReAct Agent**: Combines reasoning with tool usage for intelligent querying

## 📝 Notebook Structure
- **Cell 1**: Build and index vector database from schema
- **Cell 2**: Clean up corrupted vector DB (maintenance)
- **Cell 3**: Load existing vector database (fast start)
- **Cell 4**: Define custom vector search tools
- **Cell 5**: Create vector-powered agent
- **Cell 6**: Test vector search tools
- **Cell 7**: Test complete agent with questions
- **Cell 8**: Inspect vector store contents

---

In [1]:
# ============================================================================
# SETUP: Path Configuration for Module Imports
# ============================================================================
# Add project root to Python path to enable imports from src/ directory

import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

# Add to sys.path if not already present
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
print(f"✅ Project root added to path: {project_root}")

✅ Project root added to path: c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain


---

## 📊 Step 1: Build & Index Vector Database

This cell creates a vector database of all table schemas for semantic search.

### What This Does:
1. **Connects to Database**: Uses configured database tools
2. **Extracts Schemas**: Gets CREATE TABLE statements for all tables
3. **Fetches Sample Data**: Retrieves first 3 rows from each table (best-effort)
4. **Creates Documents**: Converts each table into a searchable document
5. **Generates Embeddings**: Converts schemas to vector representations
6. **Stores in Chroma**: Persists to disk for fast retrieval

### When to Run:
- **First time**: Initial setup
- **After schema changes**: New tables, columns, or relationships
- **Database migration**: Moving to new database
- **Corrupted vector store**: After cleanup in Cell 2

### Performance:
- Takes ~2-5 minutes for 50-100 tables
- Only needs to run once (results are persisted)

---

## 🔧 Setup: Environment Configuration

This cell configures the Python path to enable imports from the `src/` directory.

In [2]:
# ============================================================================
# CELL 1: BUILD & INDEX VECTOR DATABASE
# ============================================================================
# This cell creates a vector database of all table schemas for semantic search
# Run this once to build the index, then use Cell 3 to load existing DB

from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from src.db.db_client import db_client
from src.config.models import get_llm, get_vector_embeddings
from src.agents.tools import db_tool_manager
from src.config.vector_config import VECTOR_DB_SCHEMA_DIR, VECTOR_DB_SCHEMA_NAME
import os
import warnings
import re

# Suppress SQLAlchemy warnings about SQL Server data types
warnings.filterwarnings('ignore', message='.*Did not recognize type.*')
warnings.filterwarnings('ignore', category=Warning, module='sqlalchemy')

print("🗃️ Setting up Vector DB for database schemas...")

if True:
    print(f"⚠️ Warning: Vector DB directory '{VECTOR_DB_SCHEMA_DIR}' already exists and is not empty.")
    

    # ──────────────────────────────────────────────────────────────────────────
    # Step 1: Initialize LLM and Database Tools
    # ──────────────────────────────────────────────────────────────────────────
    llm = get_llm()
    tools = db_tool_manager.get_tools(llm=llm)

    # ──────────────────────────────────────────────────────────────────────────
    # Step 2: Identify Required SQL Tools
    # ──────────────────────────────────────────────────────────────────────────
    # We need three specific tools for schema extraction:
    # - list_all_tables: Get all table names in the database
    # - sql_db_schema: Get CREATE TABLE statements and structure
    # - sql_db_query: Execute queries to fetch sample data

    list_tables_tool = None
    schema_tool = None
    query_tool = None

    for tool in tools:
        if tool.name == "list_all_tables":
            list_tables_tool = tool
        elif tool.name == "get_table_schema":
            schema_tool = tool 
        elif tool.name == "sql_db_query":
            query_tool = tool

    if not list_tables_tool or not schema_tool:
        raise RuntimeError("❌ Required SQL tools are missing")

    # ──────────────────────────────────────────────────────────────────────────
    # Step 3: Extract and Parse Table Names
    # ──────────────────────────────────────────────────────────────────────────
    # Get raw list of tables (may be string or list format)
    table_names_str = list_tables_tool.invoke("")
    print(f"🔍 Raw output: {repr(table_names_str)}")  # Debug output

    # Handle both list and string responses
    if isinstance(table_names_str, list):
        table_names = table_names_str
    else:
        cleaned = re.sub(r'^[\[\(\{\'\"]+|[\]\)\}\'\"]+$', '', table_names_str.strip())
        # Split by comma and clean each name
        table_names = [
            name.strip().strip('()[]{}\'\"') 
            for name in cleaned.split(',')
            if name.strip() and name.strip().strip('()[]{}\'\"')
        ]

    print(f"✅ Found {len(table_names)} tables:")
    for tn in table_names:
        print(f"   • {tn}")

    # ──────────────────────────────────────────────────────────────────────────
    # Step 4: Build Document Collection from Database Schemas
    # ──────────────────────────────────────────────────────────────────────────
    # Each table becomes a Document with:
    # - page_content: Schema definition + sample data
    # - metadata: Table name, source, type

    schema_docs = []
    failed_tables = []
    success_count = 0

    for idx, table_name in enumerate(table_names, 1):
        try:
            print(f"[{idx}/{len(table_names)}] Processing: {table_name}...", end=" ")
            
            # Get CREATE TABLE statement and column definitions
            table_info = schema_tool.invoke(table_name)
            
            # Attempt to fetch sample rows (best-effort, may fail for some tables)
            sample_info = ""
            if query_tool:
                try:
                    # Get database type and use appropriate LIMIT syntax
                    db_type = db_client.db_type.lower()
                    
                    if db_type == "mssql":
                        sample_query = f"SELECT TOP 3 * FROM {table_name}"
                    elif db_type in ["oracle"]:
                        sample_query = f"SELECT * FROM {table_name} WHERE ROWNUM <= 3"
                    else:  # postgres, mysql, sqlite
                        sample_query = f"SELECT * FROM {table_name} LIMIT 3"
                    
                    sample_data = query_tool.invoke(sample_query)
                    sample_info = f"\n\nSample Data (first 3 rows):\n{sample_data}"
                    print("Sample data fetched.", sample_info)
                except Exception as exc:
                    sample_info = f"\n\nSample data not available: {exc}"
            
            # Build comprehensive document content
            content = f"""Table: {table_name}

    Schema Details:
    {table_info}
    {sample_info}

    This table can be queried to answer questions about {table_name.replace('_', ' ')}.
    """.strip()
            
            # Create Document object for vector indexing
            doc = Document(
                page_content=content,
                metadata={
                    "table_name": table_name,
                    "source": "database_schema",
                    "type": "table_schema",
                    "database_name": db_client.database,
                    "db_type": db_client.db_type,
                }
            )
            schema_docs.append(doc)
            success_count += 1
            print("✓")
        
        except Exception as exc:
            # Track failures for reporting
            error_msg = str(exc)[:150]
            failed_tables.append((table_name, error_msg))
            print(f"✗ ERROR: {error_msg}")

    # ──────────────────────────────────────────────────────────────────────────
    # Step 5: Report Processing Results
    # ──────────────────────────────────────────────────────────────────────────
    if failed_tables:
        print(f"\n⚠️ Failed Tables:")
        for table, error in failed_tables:
            print(f"   • {table}: {error}")

    print(f"\n✅ Successfully created {len(schema_docs)} schema documents")

    # ──────────────────────────────────────────────────────────────────────────
    # Step 6: Create and Persist Vector Store
    # ──────────────────────────────────────────────────────────────────────────
    # Convert documents to embeddings and store in Chroma DB
    # This enables semantic similarity search over table schemas

    embeddings = get_vector_embeddings()
    collection_name = VECTOR_DB_SCHEMA_NAME + "_" + db_client.db_type + "_" + db_client.database
    persist_directory = VECTOR_DB_SCHEMA_DIR + "/" + collection_name
    vectorstore = Chroma.from_documents(
        documents=schema_docs,
        embedding=get_vector_embeddings(),
        persist_directory=persist_directory,
        collection_name=collection_name,
    )

    print(f"✅ Vector DB ready: {len(schema_docs)} documents indexed")
    print(f"💾 Persisted to: {persist_directory}")

    # ──────────────────────────────────────────────────────────────────────────
    # Final Summary Report
    # ──────────────────────────────────────────────────────────────────────────
    print(f"\n{'='*70}")
    print("📊 PROCESSING SUMMARY")
    print(f"{'='*70}")
    print(f"✅ Successful: {success_count}/{len(table_names)}")
    print(f"❌ Failed: {len(failed_tables)}/{len(table_names)}")
    print(f"📦 Total Documents Indexed: {len(schema_docs)}")
    print(f"{'='*70}")

c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
INFO: Found 14 tables


🗃️ Setting up Vector DB for database schemas...
⚠️ Warning: Vector DB directory 'src/vectors/db_schema_vectors' already exists and is not empty.
🔍 Raw output: ['public.categories', 'public.customer_customer_demo', 'public.customer_demographics', 'public.customers', 'public.employee_territories', 'public.employees', 'public.order_details', 'public.orders', 'public.products', 'public.region', 'public.shippers', 'public.suppliers', 'public.territories', 'public.us_states']
✅ Found 14 tables:
   • public.categories
   • public.customer_customer_demo
   • public.customer_demographics
   • public.customers
   • public.employee_territories
   • public.employees
   • public.order_details
   • public.orders
   • public.products
   • public.region
   • public.shippers
   • public.suppliers
   • public.territories
   • public.us_states
[1/14] Processing: public.categories... 

INFO: Retrieved schema for: public.categories
INFO: Retrieved schema for: public.customer_customer_demo
INFO: Retrieved schema for: public.customer_demographics
INFO: Retrieved schema for: public.customers
INFO: Retrieved schema for: public.employee_territories
INFO: Retrieved schema for: public.employees
INFO: Retrieved schema for: public.order_details
INFO: Retrieved schema for: public.orders
INFO: Retrieved schema for: public.products
INFO: Retrieved schema for: public.region
INFO: Retrieved schema for: public.shippers
INFO: Retrieved schema for: public.suppliers
INFO: Retrieved schema for: public.territories
INFO: Retrieved schema for: public.us_states


Sample data fetched. 

Sample Data (first 3 rows):
[(1, 'Beverages', 'Soft drinks, coffees, teas, beers, and ales', <memory at 0x000001EACA5EBB80>), (2, 'Condiments', 'Sweet and savory sauces, relishes, spreads, and seasonings', <memory at 0x000001EACA5EB940>), (3, 'Confections', 'Desserts, candies, and sweet breads', <memory at 0x000001EACA5EBC40>)]
✓
[2/14] Processing: public.customer_customer_demo... Sample data fetched. 

Sample Data (first 3 rows):

✓
[3/14] Processing: public.customer_demographics... Sample data fetched. 

Sample Data (first 3 rows):

✓
[4/14] Processing: public.customers... Sample data fetched. 

Sample Data (first 3 rows):
[('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Sales Representative', 'Obere Str. 57', 'Berlin', None, '12209', 'Germany', '030-0074321', '030-0076545'), ('ANATR', 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Owner', 'Avda. de la Constitución 2222', 'México D.F.', None, '05021', 'Mexico', '(5) 555-4729', '(5) 555-3745'), ('ANTON

INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


✅ Vector DB ready: 14 documents indexed
💾 Persisted to: src/vectors/db_schema_vectors/db_schema_postgres_northwind

📊 PROCESSING SUMMARY
✅ Successful: 14/14
❌ Failed: 0/14
📦 Total Documents Indexed: 14


---

## 🧹 Step 2: Clean Up Vector Database (Optional Maintenance)

**⚠️ Warning**: This cell **deletes** the entire vector database. Only run when needed!

### When to Use:
- **Corrupted Database**: Errors when loading vector store
- **Schema Changes**: Major database restructuring
- **Fresh Start**: Want to rebuild from scratch
- **Troubleshooting**: Debugging vector store issues

### After Running:
- Go back to **Cell 1** to rebuild the vector database
- Rebuilding will take a few minutes

---

In [ ]:
# ============================================================================
# CELL 2: CLEAN UP VECTOR DATABASE (Optional Maintenance)
# ============================================================================
# Use this cell to remove corrupted or outdated vector database
# ⚠️ Run this ONLY when you need to rebuild from scratch

import shutil
from src.config.vector_config import VECTOR_DB_SCHEMA_DIR

# Check if vector DB exists at expected location
if os.path.exists(VECTOR_DB_SCHEMA_DIR):
    print(f"🗑️ Removing vector DB at: {VECTOR_DB_SCHEMA_DIR}")
    shutil.rmtree(VECTOR_DB_SCHEMA_DIR)
    print("✅ Vector DB removed successfully!")
    print("💡 Now run Cell 1 to rebuild it from scratch.")
else:
    print("ℹ️ No vector DB found at the expected location.")
    print(f"   Expected path: {VECTOR_DB_SCHEMA_DIR}")

---

## 📂 Step 3: Load Existing Vector Database

If you've already built the vector database in Cell 1, you can load it quickly without rebuilding.

### ✅ Benefits:
- **Fast Loading**: Loads in seconds vs. minutes to rebuild
- **Preserves Embeddings**: Uses existing vector representations
- **Normal Workflow**: Use this for day-to-day work

### When to Use:
- **Resuming Work**: Starting a new session
- **Testing**: Running queries without rebuilding
- **After Cell 1**: Once initial build is complete

### 📝 Note:
If this cell shows "No existing vector DB found", run **Cell 1** first to create it.

---

In [ ]:
# ============================================================================
# CELL 3: LOAD EXISTING VECTOR DATABASE
# ============================================================================
# Load previously created vector database instead of rebuilding from scratch
# This is much faster and should be used for normal workflow

from langchain_community.vectorstores import Chroma
from src.config.models import get_vector_embeddings
from src.config.vector_config import VECTOR_DB_SCHEMA_DIR, VECTOR_DB_SCHEMA_NAME

# Check if vector DB exists and has content
if os.path.exists(VECTOR_DB_SCHEMA_DIR) and os.listdir(VECTOR_DB_SCHEMA_DIR):
    print("ℹ️ Loading existing vector DB...")
    
    # Initialize Chroma with existing persisted data
    vector_db = Chroma(
        persist_directory=VECTOR_DB_SCHEMA_DIR,
        collection_name=VECTOR_DB_SCHEMA_NAME,
        embedding_function=get_vector_embeddings()
    )
    
    # Get document count from collection
    doc_count = vector_db._collection.count()
    print(f"✅ Loaded {doc_count} documents from existing vector DB")
    print(f"📂 Location: {VECTOR_DB_SCHEMA_DIR}")
else:
    print("⚠️ No existing vector DB found!")
    print("💡 Run Cell 1 first to create the vector database.")

---

## 🔧 Step 4: Define Vector Search Tools

Create custom LangChain tools that wrap vector store functionality for semantic table discovery.

### 🛠️ Tool 1: `semantic_table_search`
**Purpose**: Find relevant tables using natural language  
**Use Case**: "Which tables contain customer information?"  
**Returns**: Table names with relevance scores

### 🛠️ Tool 2: `get_schema_by_semantic_search`
**Purpose**: Get detailed schema for relevant tables  
**Use Case**: Need structure info before writing SQL  
**Returns**: CREATE TABLE statements and column definitions

### 🛠️ Tool 3: `list_relevant_tables_with_samples`
**Purpose**: Get complete info including sample data  
**Use Case**: Need to understand both structure and data patterns  
**Returns**: Schema + first 3 rows of data

### How It Works:
1. Agent receives natural language question
2. Converts question to vector embedding
3. Performs similarity search against stored schemas
4. Returns most relevant tables based on semantic meaning

---

In [ ]:
# ============================================================================
# CELL 4: DEFINE VECTOR SEARCH TOOLS
# ============================================================================
# Create custom LangChain tools that wrap vector store functionality
# These tools will be available to the agent for semantic table discovery

from langchain.tools import tool
from typing import List

# ──────────────────────────────────────────────────────────────────────────
# Tool 1: Semantic Table Search
# ──────────────────────────────────────────────────────────────────────────
@tool
def semantic_table_search(query: str, k: int = 3) -> str:
    """
    Find most relevant database tables using semantic search on schemas.
    
    This tool performs similarity search over vector embeddings of table schemas
    to find tables that are semantically related to the query.
    
    Args:
        query: Natural language description of what you're looking for
               Example: "sales customers orders" or "employee information"
        k: Number of top results to return (default: 3)
    
    Returns:
        String listing the most relevant tables with their similarity scores
        
    Example:
        semantic_table_search("product inventory", k=3)
        Returns top 3 tables related to products and inventory
    """
    print(f"🔍 Semantic search: '{query}'")
    
    # Perform similarity search with distance scores
    # Chroma returns (document, distance) tuples
    results_with_scores = vector_db.similarity_search_with_score(query, k=k)
    
    # Format results for agent consumption
    output_lines = [f"Top {k} tables for '{query}':"]
    for doc, score in results_with_scores:
        table_name = doc.metadata.get('table_name', 'Unknown')
        # Convert distance to similarity score (lower distance = higher similarity)
        similarity = 1.0 / (1.0 + score) if score > 0 else 1.0
        output_lines.append(f"  • {table_name} (relevance: {similarity:.3f})")
    
    return "\n".join(output_lines)


# ──────────────────────────────────────────────────────────────────────────
# Tool 2: Schema Retrieval by Semantic Search
# ──────────────────────────────────────────────────────────────────────────
@tool
def get_schema_by_semantic_search(query: str, k: int = 2) -> str:
    """
    Get detailed schema information for semantically relevant tables.
    
    Returns full CREATE TABLE statements and column definitions for tables
    that match the semantic query.
    
    Args:
        query: Natural language description of what tables you need
               Example: "tables with customer contact information"
        k: Number of tables to retrieve (default: 2)
    
    Returns:
        Detailed schema information for the most relevant tables including
        column names, types, constraints, and relationships
        
    Example:
        get_schema_by_semantic_search("employee departments", k=2)
        Returns schema details for Employee and Department tables
    """
    print(f"📋 Semantic schema retrieval: '{query}'")
    
    # Get relevant documents without scores
    docs = vector_db.similarity_search(query, k=k)
    
    if not docs:
        return "No relevant tables found for your query."
    
    # Combine schema information from multiple tables
    schema_parts = []
    for i, doc in enumerate(docs, 1):
        table_name = doc.metadata.get('table_name', 'Unknown')
        schema_parts.append(f"--- Table {i}: {table_name} ---")
        schema_parts.append(doc.page_content)
        schema_parts.append("")  # Blank line separator
    
    return "\n".join(schema_parts)


# ──────────────────────────────────────────────────────────────────────────
# Tool 3: Complete Table Information with Samples
# ──────────────────────────────────────────────────────────────────────────
@tool
def list_relevant_tables_with_samples(query: str, k: int = 2) -> str:
    """
    Get table names, schemas, AND sample data for relevant tables.
    
    This is the most comprehensive tool - it returns everything you need
    to understand a table: structure, column types, and actual data examples.
    
    Args:
        query: What you're looking for
               Example: "order history" or "product pricing"
        k: Number of tables to retrieve (default: 2)
    
    Returns:
        Complete information including:
        - Table name and metadata
        - Full schema (columns, types, constraints)
        - Sample rows (first 3 rows if available)
        
    Example:
        list_relevant_tables_with_samples("sales data", k=2)
        Returns schema + sample data for top 2 sales-related tables
    """
    print(f"📊 Getting tables with samples: '{query}'")
    
    # Retrieve semantically similar documents
    docs = vector_db.similarity_search(query, k=k)
    
    if not docs:
        return "No relevant tables found for your query."
    
    # Format comprehensive output
    result_parts = []
    for doc in docs:
        table_name = doc.metadata.get('table_name', 'Unknown')
        result_parts.append(f"\n{'='*60}")
        result_parts.append(f"Table: {table_name}")
        result_parts.append(f"{'='*60}")
        result_parts.append(doc.page_content)  # Includes schema + samples
    
    return "\n".join(result_parts)


# ──────────────────────────────────────────────────────────────────────────
# Register Tools Collection
# ──────────────────────────────────────────────────────────────────────────
VECTOR_TOOLS = [
    semantic_table_search,
    get_schema_by_semantic_search,
    list_relevant_tables_with_samples
]

print(f"✅ Created {len(VECTOR_TOOLS)} vector search tools:")
for tool in VECTOR_TOOLS:
    print(f"   • {tool.name}: {tool.description.split('.')[0]}")

---

## 🧪 Step 6: Test Vector Search Tools

Before using the full agent, let's test each vector search tool individually to verify they work correctly.

### Test Plan:
1. ✅ Search for sales-related tables
2. ✅ Search for HR/employee tables
3. ✅ Get detailed schema for products
4. ✅ Get complete info with samples

### What to Look For:
- **Relevance Scores**: Should be >0.6 for good matches
- **Table Names**: Should make semantic sense for the query
- **Schema Content**: Should include column definitions
- **Sample Data**: Should show actual values (if available)

---

In [ ]:
# ============================================================================
# CELL 6: TEST VECTOR SEARCH TOOLS
# ============================================================================
# Verify each tool works correctly before using in agent

print("🧪 Testing Vector Search Tools")
print("=" * 70)

# ──────────────────────────────────────────────────────────────────────────
# Test 1: Search for Sales Tables
# ──────────────────────────────────────────────────────────────────────────
print("\n1. 🔍 Search for sales-related tables:")
result1 = semantic_table_search.invoke({"query": "sales customers orders", "k": 3})
print(result1)

# ──────────────────────────────────────────────────────────────────────────
# Test 2: Search for Employee/HR Tables
# ──────────────────────────────────────────────────────────────────────────
print("\n2. 🔍 Search for employee/HR tables:")
result2 = semantic_table_search.invoke({"query": "employee department salary", "k": 3})
print(result2)

# ──────────────────────────────────────────────────────────────────────────
# Test 3: Get Schema for Product Tables
# ──────────────────────────────────────────────────────────────────────────
print("\n3. 📋 Get schema for product-related tables:")
result3 = get_schema_by_semantic_search.invoke({"query": "products inventory", "k": 2})
# Truncate output if too long
print(result3[:500] + "..." if len(result3) > 500 else result3)

# ──────────────────────────────────────────────────────────────────────────
# Test 4: Get Complete Information with Samples
# ──────────────────────────────────────────────────────────────────────────
print("\n4. 📊 Get tables with sample data:")
result4 = list_relevant_tables_with_samples.invoke({"query": "customer information", "k": 1})
# Truncate output if too long
print(result4[:800] + "..." if len(result4) > 800 else result4)

print("\n✅ All vector search tools tested successfully!")

---

## 🔬 Step 8: Inspect Vector Store Contents

Debug and understand what's stored in the vector database.

### What to Check:
- **Total Document Count**: Number of tables indexed
- **Sample Documents**: Preview of stored content
- **Metadata Structure**: How tables are organized
- **Embedding Model**: Which model is used
- **Storage Location**: Where files are persisted

### Use This For:
- ✅ Verify vector store was built correctly
- ✅ Debug search issues
- ✅ Understand document structure
- ✅ Check for missing tables

---

In [ ]:
# ============================================================================
# CELL 8: INSPECT VECTOR STORE CONTENTS
# ============================================================================
# Debug tool to examine what's actually stored in the vector database

print("🔍 Vector Store Contents Inspection")
print("=" * 70)

# ──────────────────────────────────────────────────────────────────────────
# Step 1: Get Collection Statistics
# ──────────────────────────────────────────────────────────────────────────
collection = vector_db._collection
total_docs = collection.count()
print(f"📊 Total documents in vector store: {total_docs}")

# ──────────────────────────────────────────────────────────────────────────
# Step 2: Retrieve Sample Documents
# ──────────────────────────────────────────────────────────────────────────
# Use generic query to get a representative sample
max_display = 5
all_docs = vector_db.similarity_search("database table", k=min(total_docs, 10))

print(f"\n📚 Showing first {min(len(all_docs), max_display)} documents:\n")

for i, doc in enumerate(all_docs[:max_display], 1):
    print(f"{'─' * 70}")
    print(f"Document {i}:")
    print(f"{'─' * 70}")
    print(f"Table: {doc.metadata.get('table_name', 'N/A')}")
    print(f"Type: {doc.metadata.get('type', 'N/A')}")
    print(f"Source: {doc.metadata.get('source', 'N/A')}")
    print(f"\nContent Preview:")
    # Show first 400 characters of document content
    content_preview = doc.page_content[:400] + "..." if len(doc.page_content) > 400 else doc.page_content
    print(content_preview)
    print()

# ──────────────────────────────────────────────────────────────────────────
# Step 3: Display Summary Statistics
# ──────────────────────────────────────────────────────────────────────────
print(f"{'='*70}")
print("📈 Vector Store Statistics:")
print(f"   • Collection Name: {VECTOR_DB_SCHEMA_NAME}")
print(f"   • Storage Location: {VECTOR_DB_SCHEMA_DIR}")
print(f"   • Total Documents: {total_docs}")
print(f"   • Embedding Model: {get_vector_embeddings().model}")
print(f"   • Documents Displayed: {min(len(all_docs), max_display)}")
print("=" * 70)

In [ ]:
# ============================================================================
# DIAGNOSTIC: Check Vector Store Status
# ============================================================================
import os
from src.config.vector_config import VECTOR_DB_SCHEMA_DIR, VECTOR_DB_SCHEMA_NAME

print("🔍 Vector Store Diagnostic\n")
print("=" * 70)

# Check 1: Does the directory exist?
print(f"\n1️⃣ Checking directory: {VECTOR_DB_SCHEMA_DIR}")
abs_path = os.path.abspath(VECTOR_DB_SCHEMA_DIR)
print(f"   Absolute path: {abs_path}")
print(f"   Exists: {os.path.exists(abs_path)}")

if os.path.exists(abs_path):
    files = os.listdir(abs_path)
    print(f"   Files: {files}")
else:
    print("   ⚠️ Directory does not exist!")
    print("   💡 Solution: Run notebook/5.1-vector-agent.ipynb Cell 1 to build vector DB")

# Check 2: Try to load the vector store
print(f"\n2️⃣ Attempting to load vector store...")
try:
    from langchain_community.vectorstores import Chroma
    from src.config.models import get_vector_embeddings
    
    vectorstore = Chroma(
        persist_directory=VECTOR_DB_SCHEMA_DIR,
        collection_name=VECTOR_DB_SCHEMA_NAME,
        embedding_function=get_vector_embeddings()
    )
    
    # Check document count
    doc_count = vectorstore._collection.count()
    print(f"   ✅ Vector store loaded successfully!")
    print(f"   📊 Total documents: {doc_count}")
    
    if doc_count == 0:
        print("   ⚠️ WARNING: Vector store is EMPTY!")
        print("   💡 Solution: Run notebook/5.1-vector-agent.ipynb Cell 1 to populate it")
    
    # Check 3: Try a sample search
    print(f"\n3️⃣ Testing sample search...")
    results = vectorstore.similarity_search("employee Department details", k=2)
    print(f"   Found {len(results)} results")
    
    if results:
        print(f"   Sample result:")
        sample = results[0]
        print(f"     • Table: {sample.metadata.get('table_name', 'Unknown')}")
        print(f"     • Content preview: {sample.page_content}...")
    
except Exception as e:
    print(f"   ❌ Error loading vector store: {e}")
    import traceback
    print("\n   Full traceback:")
    traceback.print_exc()

print("\n" + "=" * 70)
print("\n💡 If vector store is empty or missing:")
print("   1. Open notebook/5.1-vector-agent.ipynb")
print("   2. Run Cell 1 (Build & Index Vector Database)")
print("   3. Wait for it to complete (~2-5 minutes)")
print("   4. Come back and re-run this notebook")

---

## 🧬 Step 9: Test Different Embedding Models

Test and compare different embedding models for vector search.

### What Are Embeddings?
Vector representations of text that capture semantic meaning. Similar text = similar vectors.

### Available Models:
- **OpenAI**: `text-embedding-3-small`, `text-embedding-3-large`, `text-embedding-ada-002`
- **HuggingFace**: `all-MiniLM-L6-v2`, `all-mpnet-base-v2`
- **Sentence Transformers**: Various open-source models

### What This Tests:
- Current embedding model configuration
- Model loading and initialization
- Sample embedding generation
- Vector dimensionality

### 💡 Why This Matters:
- Different models have different dimensions (384, 768, 1536, 3072)
- Quality affects search accuracy
- Speed affects query latency
- Cost affects production budget

**Note**: Changing embedding models requires **rebuilding the vector store**.

In [ ]:
# Test switching embedding models
from src.config.models import get_vector_embeddings
from src.config.models import default_embedding_model

print("Current embedding model:")
print(f"  Provider: {default_embedding_model['provider']}")
print(f"  Model: {default_embedding_model['model']}")

# Get embeddings instance
embeddings = get_vector_embeddings()
print(f"\n✅ Embeddings loaded: {type(embeddings).__name__}")

# Test embedding a sample text
sample_text = "customer sales data"
embedding_vector = embeddings.embed_query(sample_text)
print(f"\n✅ Sample embedding dimension: {len(embedding_vector)}")